### Código de algoritmos

In [5]:
import numpy as np
import scipy.linalg as sl
""""
def gauss_jordan(A): # retorna la inversa de la matriz A
    nrow = np.shape(A)[0] # numero de filas de A
    
    i= 1
    invA = np.identity(nrow)    #inicializa invA
    while i < nrow:
        a = np.array(A[:,i]/A[i,i]).reshape(nrow, 1)
        a[i] = 1-1/A[i,i]
        e = np.zeros((1, nrow))
        e[0,i] = 1
        
        L = np.identity(nrow) - np.dot(a, e)
        invA = np.dot(L, invA)
        if np.allclose(np.dot(invA, A),np.identity(nrow)):
            return invA
        i += 1
    print("La matriz :\n{}\n\nno es inversible".format(A))
"""

def max_norm(A):    # calcula la norma del maximo de la matriz A
    sum_row = sum(np.abs(A[0,:]))
    nrow = np.shape(A)[0]
    for i in range(1,nrow):
        s = sum(np.abs(A[i,:]))
        if s > sum_row:
            sum_row = s
    
    return sum_row


def cond(A):    # calcula la condicional de la matriz A
    invA = sl.inv(A) #DEPURAR GAUSS JORDAN , usando modulo scipy.linalg

    return max_norm(A)*max_norm(invA)

"""
tener cuidado con el tipo de datos del array(cantidades pequeñas lo aproxima a 0 
y asi para matrices no triangulares retorna True 
"""

# verifica si una matriz es triangular inferior teniendo en cuenta que los elementos bajo la diagonal difieren a lo sumo ERROR=1e-7 de cero
def istriangular_sup(A):   
    nrow = np.shape(A)[0]

    for k in range(0, nrow):
        if not np.allclose(A[k+1:, k] ,0.0) or  not np.isclose(A[nrow-1, nrow-1], 0.0):
            return False
    return True

def sup_triang(A):
    nrow=np.shape(A)[0]

    for i in range(nrow) :
        for j in range (nrow) :
            if i>j :
                if A[i,j]>1e-7 :
                    return False
    return True
# depurar
def eliminacion_gaussiana(A, b):
    nrow = np.shape(A)[0]
    
    i=0
    while not sup_triang(A) and i < nrow:
        #inicializa los vectores a y e
        a = np.zeros((nrow, 1))
        a[i+1:, i] = A[i+1:, i]/A[i,i]
        e = np.zeros((1, nrow))
        e[0, i] = 1

        L = np.identity(nrow) - np.dot(a, e)
        A = np.dot(L, A)
        b = np.dot(L, b)

        i+=1
    return A,b



def solve(A, b):
    #tA, tb: matrices obtenidas  despues de aplicar la transformacion gaussiana
    tA, tb = eliminacion_gaussiana(A, b)
    nrow = np.shape(A)[0]
    x = np.zeros(nrow)

    # resulve el sistema triangular superior (tA)x = tb, desde abajo hacia arriba
    for i in range(nrow-1, -1, -1): # range(nrow-1, -1, -1) = n-1, n-2, n-3, ..., 1, 0
        s = sum(tA[i,j]*x[j] for j in range(i+1, nrow))
        x[i] = (tb[i]- s)/tA[i,i] 
    
    return x

#depurar
def error_relativo(A, b):
    x0 = solve(A, b)    # es la aproximacion de la solucion sel sistema Ax=b, resuelto mediante eliminacion gaussiana
    r = np.dot(A, x0) - b   #vector residuo
    
    cota_sup = cond(A) * max_norm(r)/max_norm(b)
    cota_inf = max_norm(r)/(max_norm(b)*cond(A))

    print("\n\nEl error relativo de la aproximacion({}) esta entre:\n{}\n{}".format(x0, cota_inf, cota_sup)) 

#depurar
def refinamiento_iterativos(A, b, MAX_ITERATIONS=10):
    x0 = solve(A, b)    # aproximacion de la solucion del sistema Ax=b
    invA =sl.inv(A)  #aproximacion de la inversa de A
    print("k | xk")
    print("---------------")
    print("{} | \t{}".format(0,x0))
    for k in range(MAX_ITERATIONS) :
        x=x0 + np.dot(invA,b-np.dot(A,x0))
        print("{} | \t{} ".format(k,x))
        x0=x

### Cálculos 
Dado el siguiente el siguiente sistema de 


\begin{bmatrix}
    0.9  & -0.25 & -0.2 & -0.15\\
    -0.3 &  0.6  & -0.5 & -0.2\\
    -0.1 & -0.15 & 0.9  & -0.16\\
    -0.4 & -0.3  &-0.2  & 0.7
\end{bmatrix}
\begin{bmatrix}
    x_1\\ x_2\\ x_3\\ x_4
\end{bmatrix}
$=$
\begin{bmatrix}
    5000\\ 7500\\125000\\70000
\end{bmatrix}


In [23]:
"""
x0: Aproximación de la solución del sistema Ax=b mediante el método de eliminación Gaussiana.
Er: Error relativo al calcular x0.
cond(A): Condición de A.
B : Matriz aproximación de la inversa de A.
xk: Solución mejorada, luego de aplicar k refinamiento a x0.
tA,tb: Matrices resultantes luego de aplicar la transformación gaussiana.
"""
A=np.array([0.9,-0.25,-0.2,-0.15,-0.3,0.6,-0.5,-0.2,-0.1,
             -0.15,0.9,-0.16,-0.4,-0.3,-0.2,0.7],dtype=np.float128).reshape(4,4)
b=np.array([5000,7500,125000,7000],dtype=np.float128).reshape(4,1)


# Cálculo de la solución aproximada
x0=solve(A,b)
print("La solución aproximada del sistema es:\t{}".format(x0))


#Eliminación gaussiana
tA,tb=eliminacion_gaussiana(A,b)
print(" Matriz triangular (tA) \n{}\n \n(tb)\n {}".format(tA,tb))



#Cálculo de la inversa con gauss-jordan
import scipy.linalg as sl 
invA=sl.inv(A)
print("\n\nLa inversa aproximada de A es :\n\n{}".format(invA))


# Cálculo de la condicional de la matriz
 
print("\n\n La condicional de la matriz A es :\t{}\n".format(cond(A)))




# Error relativo
error_relativo(A, b)


#Refinamiento de la solucion btenida de por el metodo de eliminacion gausiana 

refinamiento_iterativos(A,b)


La solución aproximada del sistema es:	[ 75441.35802469 133055.55555556 140666.66666667  10000.        ]
 Matriz triangular (tA) 
[[ 0.9  -0.25 -0.2  -0.15]
 [-0.3   0.6  -0.5  -0.2 ]
 [-0.1  -0.15  0.9  -0.16]
 [-0.4  -0.3  -0.2   0.7 ]]
 
(tb)
 [[  5000.]
 [  7500.]
 [125000.]
 [  7000.]]


La inversa aproximada de A es :

[[4.67409588 4.73296888 4.41547519 3.36312027]
 [6.77880572 9.55424727 8.15811606 6.0470984 ]
 [2.78174937 3.50504626 4.24726661 2.56833474]
 [6.37089992 7.80067283 7.23296888 6.67577796]]


 La condicional de la matriz A es :	48.8612279226241

El error relativo de la aproximacion([ 75441.35802469 133055.55555556 140666.66666667  10000.        ]) esta entre:
0.08249672653351958
196.95430137578023
k | xk
---------------
0 | 	[ 75441.35802469 133055.55555556 140666.66666667  10000.        ]
0 | 	[[  623857.04347465  1027459.95409308  -900548.78438152  2212132.36107218]
 [ 1090395.10066557  1762818.14004922 -1669093.92683965  3963545.65513088]
 [  599012.63640989   92